In [10]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('../data/misc/daily_portfolio_value.csv')
df.head()

,date,portfolio_value,ADA-USD,ETH-USD,LINK-USD,LTC-USD,SOL-USD,UNI-USD,XRP-USD,ADI,...,LRCX,MCHP,MCO,MOH,ODFL,PCG,PHM,PLD,STZ,TSLA
0,2025-03-01,1.000000e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2025-03-02,1.000000e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2025-03-03,9.999924e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2025-03-04,9.992645e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2025-03-05,1.000049e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
trades = pd.read_csv('../data/misc/trade_log.csv')
trades.head()

,asset,date,action,price,units,signed_value
0,LII,2025-03-03,BUY_NEW,586.979980,5.195480e+01,-30496.424882
1,LII,2025-03-06,BUY_ADD,583.900024,2.774973e-01,-162.030664
2,UNI-USD,2025-03-10,BUY_NEW,0.000158,5.797783e+06,-916.049651
3,ADI,2025-03-10,BUY_NEW,216.259995,1.330952e+02,-28783.165230
4,ADI,2025-03-12,BUY_ADD,206.619995,5.814890e+00,-1201.472533


In [19]:
## Annual Return
df['date'] = pd.to_datetime(df['date'])

# Compute number of days in testing period
num_days = len(df)

# Compute total and annualised return
testing_return = df['portfolio_value'].iloc[-1] / df['portfolio_value'].iloc[0] - 1

if num_days > 0:
    annualised_return = (1 + testing_return) ** (365 / num_days) - 1
    print(f"Testing period return: {testing_return:.4%}")
    print(f"Annualised return: {annualised_return:.4%}")


Testing period return: -0.4731%
Annualised return: -5.4305%


In [14]:
## Sharpe Ratio
def sharpe_ratio(return_series, N, rf):
    mean = return_series.mean() * N - rf
    sigma = return_series.std() * np.sqrt(N)
    return mean/sigma

sharpe_ratio(df['portfolio_value'].pct_change(), 252, 0.0431)

0.19034691718842298

In [15]:
# Max Drawdown
peak = df['portfolio_value'].cummax()
drawdown = (df['portfolio_value'] - peak) / peak
f'{drawdown.min() * 100:.2f}%'

'-10.56%'

In [7]:
# Win Rate
d = {}
wins, losses = 0, 0
for row in trades.iterrows():
    entry = row[1]

    if entry['action'] == 'BUY_NEW':
        d[entry['asset']] = {
            'date': entry['date'],
            'price': entry['price'],
            'units': entry['units']
        }
        
    if entry['action'] == 'BUY_ADD':
        assert entry['asset'] in d
        d[entry['asset']]['price'] = (
                d[entry['asset']]['price'] * d[entry['asset']]['units'] + entry['price'] * entry['units']
            ) / (
                d[entry['asset']]['units'] + entry['units']
            )
        d[entry['asset']]['units'] += entry['units']
        
    if entry['action'] in ['SELL_SL', 'SELL_TP', 'SELL_SHORT']:
        assert entry['asset'] in d
        if entry['price'] > d[entry['asset']]['price']:
            wins += 1
        elif entry['price'] < d[entry['asset']]['price']:
            losses += 1
                     
f'{wins / (wins + losses) * 100:.2f}%'

'40.00%'

In [8]:
losses

3

In [9]:
leftover_value = df.iloc[-1][d.keys()]
leftover_value

LII                 0.0
UNI-USD      819.434388
ADI        28628.046387
MCHP       36139.079876
MOH                 0.0
XRP-USD             0.0
Name: 30, dtype: object

In [10]:
# Annual Volatility
pct_change = df['portfolio_value'].pct_change()[1:]
f'{pct_change.std() * (252 ** 0.5) * 100:.2f}%'

'65.86%'

In [11]:
# Drawdown Patterns
is_drawdown = drawdown < 0
(is_drawdown & (~is_drawdown.shift(1).fillna(False))).sum()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_49160\226863221.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  (is_drawdown & (~is_drawdown.shift(1).fillna(False))).sum()


3

In [12]:
# Number of Trades
len(trades)

18

In [20]:
eps = 1e-6
d = {}
holding_periods = []

for _, entry in trades.iterrows():
    asset = entry['asset']
    date = pd.to_datetime(entry['date'])  # ensure datetime

    if entry['action'] == 'BUY_NEW':
        d[asset] = {'date': date, 'units': entry['units']}

    elif entry['action'] == 'BUY_ADD':
        assert asset in d
        d[asset]['units'] += entry['units']

    elif entry['action'] in ['SELL_SL', 'SELL_TP', 'SELL_SHORT']:
        assert asset in d
        d[asset]['units'] -= entry['units']
        
        if d[asset]['units'] < eps:
            dt = date - d[asset]['date']  # both are datetime now
            holding_periods.append(dt.days)
            del d[asset]

last_date = df['date'].iloc[-1]

# For open positions, measure to last_date:
holding_periods += [
    (last_date - data['date']).days
    for asset, data in d.items()
]

# Average holding period
average_holding_period = sum(holding_periods) / len(holding_periods)
average_holding_period


8.125

In [15]:
# Profit per Trade

(df.iloc[-1]['portfolio_value'] - df.iloc[0]['portfolio_value']) / len(trades)


-262.8303122336446

In [16]:
# Profit per Trade (Alternative)

'''
I will consider the price at the end of the trading period 
of the assets not sold as part of the profit.
'''

leftover_value = df.iloc[-1][d.keys()]
leftover_value = leftover_value[leftover_value > 0]

pd.concat([trades['signed_value'], leftover_value]).mean()

-203.8271820423475